In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:110: SyntaxWarning: invalid escape sequence '\i'
  """Returns the m \in {-l,...,l} indices"""
/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:116: SyntaxWarning: invalid escape sequence '\i'
  """Returns the 2D outerproduct of m_i \in {-l_i,... , l_i} and m_j \in {-l_j,... , l_j} to index the (l_i, l_j) block of the hamiltonian


In [2]:
import numpy as np 
import torch 
import metatensor.torch as mts
from metatensor.torch import TensorMap, Labels, TensorBlock
import ase 
from mlelec.data.dataset import QMDataset
from mlelec.utils.target_utils import get_targets
from mlelec.utils.twocenter_utils import _to_coupled_basis

# print(torch.cuda.is_available())

/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:110: SyntaxWarning: invalid escape sequence '\i'
  """Returns the m \in {-l,...,l} indices"""
/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:116: SyntaxWarning: invalid escape sequence '\i'
  """Returns the 2D outerproduct of m_i \in {-l_i,... , l_i} and m_j \in {-l_j,... , l_j} to index the (l_i, l_j) block of the hamiltonian
/Users/jigyasa/scratch/mlelec/src/mlelec/utils/pbc_utils.py:1688: SyntaxWarning: invalid escape sequence '\d'
  assert np.isclose(torch.norm(matrixT - matrixmT.T).item(), 0.0), f"Failed to check H({T}) = H({mT})^\dagger"
/Users/jigyasa/miniconda3/lib/python3.12/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True

In [3]:
frames = [ase.Atoms('H2O', positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], pbc = False)]
qmdata = QMDataset(frames = frames, 
                   fock_realspace= torch.randn(1,7,7),
                   dimension = 0, 
                   orbs = {8:[[1,0,0],[2,0,0],[2,1,0], [2,1,1], [2,1,-1]], 1:[[1,0,0]]},
                   orbs_name = 'sto-3g',    
                   device = 'cpu'
)   
blocks, coupled_blocks  = get_targets(qmdata, cutoff = 4, device = 'cpu', all_pairs = False, sort_orbs =True, return_uncoupled=True)
# just needed them for keys 

/Users/jigyasa/scratch/mlelec/src/mlelec/data/dataset.py:210: UserWarning: Overlap matrices not provided
  warnings.warn("Overlap matrices not provided")


In [69]:
blocks[0].values.shape

torch.Size([1, 1, 1, 1])

For a 3D tensor, we should have the same bloch structure, except, we have an additional components dimension 

In [18]:
blocks_3D = []
position_components = Labels(['m_3'], values = torch.tensor([-1,0,1]).reshape(3,-1))
for block in blocks:
    nsample, nmi, nmj, nprop = block.values.shape
    blocks_3D.append(
        TensorBlock( values = torch.randn(nsample, nmi, nmj,3, nprop), 
                    components = [block.components[0],  block.components[1], position_components],
                    properties = block.properties,
                    samples = block.samples,
        )
    )
key_names = blocks.keys.names + ["l_3"]
key_value = torch.nn.functional.pad(blocks.keys.values, (0,1), mode='constant', value=1) 

uncoupled_blocks_3D = TensorMap( Labels(key_names, key_value) , blocks_3D)

uncoupled_blocks_3D = mts.remove_dimension(uncoupled_blocks_3D, 'samples', 'cell_shift_a') 
uncoupled_blocks_3D = mts.remove_dimension(uncoupled_blocks_3D, 'samples', 'cell_shift_b') 
uncoupled_blocks_3D = mts.remove_dimension(uncoupled_blocks_3D, 'samples', 'cell_shift_c') 


In [19]:
uncoupled_blocks_3D

TensorMap with 12 blocks
keys: block_type  species_i  n_i  l_i  species_j  n_j  l_j  l_3
          -1          1       1    0       1       1    0    1
          0           1       1    0       1       1    0    1
          0           8       1    0       8       1    0    1
          0           8       1    0       8       2    0    1
          0           8       1    0       8       2    1    1
          0           8       2    0       8       2    0    1
          0           8       2    0       8       2    1    1
          0           8       2    1       8       2    1    1
          1           1       1    0       1       1    0    1
          2           1       1    0       8       1    0    1
          2           1       1    0       8       2    0    1
          2           1       1    0       8       2    1    1

In [20]:
blocks_2Dx = []; blocks_2Dy = []; blocks_2Dz = []
for block in uncoupled_blocks_3D:
    nsample, nmi, nmj, _, nprop = block.values.shape
    blocks_2Dy.append(
        TensorBlock( values = block.values[...,0,:], 
                    components = [block.components[0],  block.components[1]],
                    properties = block.properties,
                      samples = block.samples,
        ))
    blocks_2Dz.append(
        TensorBlock( values = block.values[...,1,:], 
                    components = [block.components[0],  block.components[1]],
                    properties = block.properties,
                    samples = block.samples,
        ))
    blocks_2Dx.append(
        TensorBlock( values = block.values[...,2,:], 
                    components = [block.components[0],  block.components[1]],
                    properties = block.properties,
                    samples = block.samples,
        ))
uncoupled_blocks_2Dx = TensorMap( blocks.keys , blocks_2Dx)
uncoupled_blocks_2Dy = TensorMap( blocks.keys , blocks_2Dy)
uncoupled_blocks_2Dz = TensorMap( blocks.keys , blocks_2Dz)

# uncoupled_blocks_2Dx = mts.remove_dimension(uncoupled_blocks_2Dx, 'samples', 'cell_shift_a') 
# uncoupled_blocks_2Dx = mts.remove_dimension(uncoupled_blocks_2Dx, 'samples', 'cell_shift_b') 
# uncoupled_blocks_2Dx = mts.remove_dimension(uncoupled_blocks_2Dx, 'samples', 'cell_shift_c') 

# uncoupled_blocks_2Dy = mts.remove_dimension(uncoupled_blocks_2Dy, 'samples', 'cell_shift_a') 
# uncoupled_blocks_2Dy = mts.remove_dimension(uncoupled_blocks_2Dy, 'samples', 'cell_shift_b') 
# uncoupled_blocks_2Dy = mts.remove_dimension(uncoupled_blocks_2Dy, 'samples', 'cell_shift_c') 

# uncoupled_blocks_2Dz = mts.remove_dimension(uncoupled_blocks_2Dz, 'samples', 'cell_shift_a') 
# uncoupled_blocks_2Dz = mts.remove_dimension(uncoupled_blocks_2Dz, 'samples', 'cell_shift_b') 
# uncoupled_blocks_2Dz = mts.remove_dimension(uncoupled_blocks_2Dz, 'samples', 'cell_shift_c') 

In [6]:
coupled_blocks_2Dx = _to_coupled_basis(uncoupled_blocks_2Dx, skip_symmetry = False, device = 'cpu', translations = None)
coupled_blocks_2Dy = _to_coupled_basis(uncoupled_blocks_2Dy, skip_symmetry = False, device = 'cpu', translations = None)
coupled_blocks_2Dz = _to_coupled_basis(uncoupled_blocks_2Dz, skip_symmetry = False, device = 'cpu', translations = None)

create a tensor map from these three individually coupled tmaps, concatenating them along the components dim

In [7]:
coupled_blocks_2Dx = coupled_blocks_2Dx.keys_to_properties(['species_i', 'n_i', 'l_i','species_j', 'n_j', 'l_j'])
coupled_blocks_2Dy = coupled_blocks_2Dy.keys_to_properties(['species_i', 'n_i', 'l_i','species_j', 'n_j', 'l_j'])
coupled_blocks_2Dz = coupled_blocks_2Dz.keys_to_properties(['species_i', 'n_i', 'l_i','species_j', 'n_j', 'l_j'])

In [8]:
uncoupled_2 = []
for (bx,by,bz) in zip(coupled_blocks_2Dy.blocks(), coupled_blocks_2Dz.blocks(),coupled_blocks_2Dx.blocks()):
    ## Assert that the blocks correspond to the same keys
    assert bx.values.shape == by.values.shape == bz.values.shape
    uncoupled_2.append( TensorBlock( values = torch.stack([bx.values, by.values,bz.values], dim=2), 
                                    components = [bx.components[0],position_components], 
                                    samples = bx.samples, 
                                    properties = bx.properties
    ) )
newkeys = Labels(coupled_blocks_2Dz.keys.names+['L2'], values = torch.nn.functional.pad(coupled_blocks_2Dz.keys.values, (0,1), mode='constant', value=1))
uncoupled_2 = TensorMap(newkeys, uncoupled_2)

In [9]:
uncoupled_2

TensorMap with 6 blocks
keys: block_type  L  L2
          0       0  1
          0       1  1
          0       2  1
          1       0  1
          2       0  1
          2       1  1

In [77]:
coupled_blocks = _to_coupled_basis(uncoupled_blocks_3D, skip_symmetry = False, device = 'cpu', translations = None, high_rank = True)

/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:110: SyntaxWarning: invalid escape sequence '\i'
  """Returns the m \in {-l,...,l} indices"""
/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:116: SyntaxWarning: invalid escape sequence '\i'
  """Returns the 2D outerproduct of m_i \in {-l_i,... , l_i} and m_j \in {-l_j,... , l_j} to index the (l_i, l_j) block of the hamiltonian


In [82]:
coupled_blocks[0].values

tensor([[[-0.8360],
         [ 1.1375],
         [ 0.0626]]])

In [64]:
uncoupled_blocks_3D[0].samples.values.shape

torch.Size([1, 3])

In [57]:
coupled_blocks

{(0, 1, 0, 1): {1: tensor([[[-0.8360,  1.1375,  0.0626]]])}}

In [22]:
from mlelec.utils.symmetry import ClebschGordanReal
CG = ClebschGordanReal(10, 'cpu')

In [23]:
def couple(decoupled, iterate = 0, cg=None, selfdevice= 'cpu', lmax=10):
       
        coupled = {}

        # when called on a matrix, turns it into a dict form to which we can
        # apply the generic algorithm
        if not isinstance(decoupled, dict):
            l2 = (decoupled.shape[-1] - 1) // 2
            decoupled = {(): {l2: decoupled}}

        # runs over the tuple of (partly) decoupled terms
        for ltuple, lcomponents in decoupled.items():
            # each is a list of L terms
            for lc in lcomponents.keys():
                # this is the actual matrix-valued coupled term,
                # of shape (..., 2l1+1, 2l2+1), transforming as Y^m1_l1 Y^m2_l2
                dec_term = lcomponents[lc]
                l1 = (dec_term.shape[-2] - 1) // 2
                l2 = (dec_term.shape[-1] - 1) // 2

                # there is a certain redundance: the L value is also the last entry
                # in ltuple
                if lc != l2:
                    raise ValueError(
                        "Inconsistent shape for coupled angular momentum block."
                    )

                # in the new coupled term, prepend (l1,l2) to the existing label
                device = dec_term.device
                if device != selfdevice:
                    dec_term = dec_term.to(selfdevice)
                
                coupled[(l1, l2) + ltuple] = {}
                for L in range(
                    max(l1, l2) - min(l1, l2), min(lmax, (l1 + l2)) + 1
                ):
                    # Lterm = torch.einsum('spmn,mnM->spM', dec_term, self._cg[(l1, l2, L)])
                    coupled[(l1, l2) + ltuple][L] = torch.tensordot(dec_term, cg._cg[(l1, l2, L)].to(dec_term), dims=2)

        # repeat if required
        if iterate > 0:
            coupled = couple(coupled, iterate - 1, cg= cg, selfdevice=selfdevice,lmax=lmax)
        return coupled

In [73]:
a = torch.randn(1,5,7,3)


/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:110: SyntaxWarning: invalid escape sequence '\i'
  """Returns the m \in {-l,...,l} indices"""
/Users/jigyasa/scratch/mlelec/src/mlelec/utils/twocenter_utils.py:116: SyntaxWarning: invalid escape sequence '\i'
  """Returns the 2D outerproduct of m_i \in {-l_i,... , l_i} and m_j \in {-l_j,... , l_j} to index the (l_i, l_j) block of the hamiltonian


In [75]:
torch.moveaxis(a, -1,-3).shape

torch.Size([1, 3, 5, 7])

In [54]:
a = couple(a, 1,cg = CG,)

In [55]:
coupled = a
for coupledkey in coupled:
    k = coupledkey[1]
    print(coupledkey)
    for L in coupled[coupledkey]:
        print(coupledkey, L)
                    # block_idx = tuple(idx) + (k, L)
                    # skip blocks that are zero because of symmetry - TBD 
                    # if ai == aj and ni == nj and li == lj:
                    #     parity = (-1) ** (li + lj + L)
                    #     if ((parity == -1 and block_type in (0, 1)) or (parity == 1 and block_type == -1)) and not skip_symmetry:
                    #         continue
                    
        print(block.samples.values.shape, coupled[coupledkey][L].shape, torch.moveaxis(coupled[coupledkey][L], -1, -2).shape)


(2, 2, 3, 1)
(2, 2, 3, 1) 0
torch.Size([2, 3]) torch.Size([1, 1]) torch.Size([1, 1])
(2, 2, 3, 1) 1
torch.Size([2, 3]) torch.Size([1, 3]) torch.Size([3, 1])
(2, 2, 3, 1) 2
torch.Size([2, 3]) torch.Size([1, 5]) torch.Size([5, 1])
(2, 2, 3, 1) 3
torch.Size([2, 3]) torch.Size([1, 7]) torch.Size([7, 1])
(2, 2, 3, 1) 4
torch.Size([2, 3]) torch.Size([1, 9]) torch.Size([9, 1])
(2, 3, 3, 1)
(2, 3, 3, 1) 1
torch.Size([2, 3]) torch.Size([1, 3]) torch.Size([3, 1])
(2, 3, 3, 1) 2
torch.Size([2, 3]) torch.Size([1, 5]) torch.Size([5, 1])
(2, 3, 3, 1) 3
torch.Size([2, 3]) torch.Size([1, 7]) torch.Size([7, 1])
(2, 3, 3, 1) 4
torch.Size([2, 3]) torch.Size([1, 9]) torch.Size([9, 1])
(2, 3, 3, 1) 5
torch.Size([2, 3]) torch.Size([1, 11]) torch.Size([11, 1])
(2, 4, 3, 1)
(2, 4, 3, 1) 2
torch.Size([2, 3]) torch.Size([1, 5]) torch.Size([5, 1])
(2, 4, 3, 1) 3
torch.Size([2, 3]) torch.Size([1, 7]) torch.Size([7, 1])
(2, 4, 3, 1) 4
torch.Size([2, 3]) torch.Size([1, 9]) torch.Size([9, 1])
(2, 4, 3, 1) 5
torch.Si